In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

In [2]:
# GPU/CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
# 데이터 불러오기, 전처리
transforms = transforms.Compose([
                                 transforms.Resize(64), 
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 모델 불러오기
model = torchvision.models.alexnet(pretrained=True)

model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
# out_feautres: 1000 -> 10
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_ftrs, 10)
model = model.to(device)

model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# 모델 프리징

## 파라미터 번호 확인하기, feature_extraction: 0~9 -> 9번째 변수까지 backpropagation을 비활성화
for i, (name, param) in enumerate(model.named_parameters()):
    
    print(i, name)

    param.requires_grad = False
    if i == 9:
        print('end')
        break

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
end


In [7]:
# requires_grad 확인
print(model.features[0].weight.requires_grad)
print(model.features[0].bias.requires_grad)
print(model.features[3].weight.requires_grad)
print(model.features[3].bias.requires_grad)
print(model.features[6].weight.requires_grad)
print(model.features[6].bias.requires_grad)
print(model.features[8].weight.requires_grad)
print(model.features[8].bias.requires_grad)
print(model.features[10].weight.requires_grad)
print(model.features[10].bias.requires_grad)
print(model.classifier[1].weight.requires_grad)
print(model.classifier[1].bias.requires_grad)
print(model.classifier[4].weight.requires_grad)
print(model.classifier[4].bias.requires_grad)
print(model.classifier[6].weight.requires_grad)
print(model.classifier[6].bias.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [8]:
# 손실 함수 최적화 기법
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

In [9]:
# 모델 학습
n_batch = len(train_loader)

for epoch in range(3):

    running_loss = 0.0

    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    cost = running_loss/n_batch
    print('[%d] loss:%.3f'%(epoch+1, cost))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1] loss:1.232
[2] loss:1.157
[3] loss:1.142


In [10]:
# 모델 저장
PATH = './cifar_alexnet.pth'
torch.save(model.state_dict(), PATH)

In [13]:
# 모델 불러오기
model = torchvision.models.alexnet(pretrained=False)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_ftrs, 10)
model = model.to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [14]:
# 모델 평가
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)

        total += len(labels)

        outputs = model(images)
        predicted = torch.max(outputs, axis=1)[1]
        correct += (predicted == labels).sum().item()
        
print(f"Accuracy: {100*correct/total}")

Accuracy: 60.24
